In [ ]:
import pickle

from math import sqrt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scikit_posthocs as posthocs

picklepath = '../mainexperiment-results/results.pickle'
cols = ['zoning', 'edge_maps', 'zoning_chain_code', 'local_binary_pattern'] # column names for box plots

cmap = ['1', '#fb6a4a',  '#08306b',  '#4292c6', '#c6dbef']
heatmap_args = {'cmap': cmap, 'linewidths': 0.25, 'linecolor': '0.5', 'clip_on': False, 'square': True, 'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]}

with open(picklepath, 'rb') as f:
    results = pickle.load(f)

In [ ]:
def tests_for_normality(accuracy_results):
    """ Perfroms Shapiro-Wilk test for checking if all accuracy results 
        (for every feature extraction method) obtained using one classifier 
        come from normal distribution. 
        
        :params 
            accuracy_results dictionary 
                feature_extraction_method: 1 x N array
            
        :returns 
            list with p values for each sample """
    
    p_values = []
    for res in list(accuracy_results.values()):
        p = stats.shapiro(res)
        p_values.append(p[1])
    
    return np.array(p_values)

In [ ]:
def calculate_mean_and_se(samples_from_classifier):
    """ Calculates mean and 0.95 SE from sample (feature extraction method) obtained with one classifier 
    
    :params
        sample_from_classifier dictionary where key - feature extraction method, value - list of sample results 
    
    :returns
        stats array where row corresponds any feature exctraction method and column[0] mean and column[1] 0.95 SE """

    stats = np.zeros((4,2))
    for k, (feature_extraction_method, sample_results) in enumerate(samples_from_classifier.items()):
        mu = np.mean(sample_results)
        se = np.std(sample_results) / sqrt(len(sample_results)) * 0.95
        stats[k, 0] = mu
        stats[k, 1] = se
    
    return stats

In [ ]:
def show_mean_se_plot(clf_results):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)

    minor_ticks = np.linspace(0.0, 1.0, 21)
    ax.set_yticks(minor_ticks, minor=True)

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.grid(True)

    plt.errorbar(x=cols, y=clf_results[:,0], yerr=clf_results[:,1], fmt='s')

In [ ]:
# means for groups 
for clf, fem_results in results.items():
    print()
    for feature_extraction_methods, result in fem_results.items():
        res = np.array(result)
        print(clf, feature_extraction_methods, np.mean(res))

In [ ]:
# test for normality of each sample SVM
svm_results = results['svm']
svm_p_values = tests_for_normality(svm_results)

print("All samples evaluated with svm come from normal distribution: ", np.all(svm_p_values > 0.05))

In [ ]:
svm_stats = calculate_mean_and_se(svm_results)
show_mean_se_plot(svm_stats)

In [ ]:
data = np.array([svm_results[cols[0]], 
                 svm_results[cols[1]],
                 svm_results[cols[2]],
                 svm_results[cols[3]]])

df = pd.DataFrame(data.T, columns=cols)
plt.figure(figsize=(10, 5))
df.boxplot()
plt.savefig('svm-boxplot.eps', format='eps', dpi=1000)

In [ ]:
# Kruskal-Wallis non-parametric test for SVM
stats.mstats.kruskalwallis(svm_results[cols[0]],
                           svm_results[cols[1]],
                           svm_results[cols[2]],
                           svm_results[cols[3]])

In [ ]:
# Kurskal-Wallis test rejected null hypothesis
# post hoc test for SVM
df_melt = pd.melt(df, value_vars=cols)
svm_posthoc_results = posthocs.posthoc_conover(df_melt, val_col='value', group_col='variable', p_adjust='holm')

# Format: diagonal, non-significant, p<0.001, p<0.01, p<0.05
posthocs.sign_plot(svm_posthoc_results, **heatmap_args)

In [ ]:
# test for normality of each sample KNN
knn_results = results['knn']
knn_p_values = tests_for_normality(knn_results)

print("All samples evaluated with knn come from normal distribution: ", np.all(knn_p_values > 0.05))

In [ ]:
knn_stats = calculate_mean_and_se(knn_results)
show_mean_se_plot(knn_stats)

In [ ]:
data = np.array([knn_results[cols[0]], 
                 knn_results[cols[1]],
                 knn_results[cols[2]],
                 knn_results[cols[3]]])

df = pd.DataFrame(data.T, columns=cols)
plt.figure(figsize=(10, 5))
df.boxplot()
plt.savefig('knn-boxplot.eps', format='eps', dpi=1000)

In [ ]:
# one way ANOVA for KNN
stats.f_oneway(knn_results[cols[0]],
               knn_results[cols[1]],
               knn_results[cols[2]],
               knn_results[cols[3]])

In [ ]:
# test for equality of variances of each sample KNN
stats.bartlett(knn_results[cols[0]],
               knn_results[cols[1]],
               knn_results[cols[2]],
               knn_results[cols[3]])

In [ ]:
# post hoc test for KNN
# bartlett test does not reject null hypothesis, hence we can use pairwise ttest
df_melt = pd.melt(df, value_vars=cols)
knn_posthoc_results = posthocs.posthoc_ttest(df_melt, val_col='value', group_col='variable', p_adjust='holm')

# Format: diagonal, non-significant, p<0.001, p<0.01, p<0.05
posthocs.sign_plot(knn_posthoc_results, **heatmap_args)

In [ ]:
# test for normality of each sample MLP
mlp_results = results['mlp']
mlp_p_values = tests_for_normality(mlp_results)

print("All samples evaluated with mlp come from normal distribution: ", np.all(mlp_p_values > 0.05))

In [ ]:
mlp_stats = calculate_mean_and_se(mlp_results)
show_mean_se_plot(mlp_stats)

In [ ]:
data = np.array([mlp_results[cols[0]], 
                 mlp_results[cols[1]],
                 mlp_results[cols[2]],
                 mlp_results[cols[3]]])

df = pd.DataFrame(data.T, columns=cols)
plt.figure(figsize=(10, 5))
df.boxplot()
plt.savefig('mlp-boxplot.eps', format='eps', dpi=1000)

In [ ]:
# one way ANOVA for MLP
stats.f_oneway(mlp_results[cols[0]], 
               mlp_results[cols[1]],
               mlp_results[cols[2]],
               mlp_results[cols[3]])

In [ ]:
# test for equality of variances of each sample MLP
stats.bartlett(mlp_results[cols[0]], 
               mlp_results[cols[1]],
               mlp_results[cols[2]],
               mlp_results[cols[3]])

In [ ]:
# post hoc test for MLP
# bartlett test does not reject null hypothesis, hence we can use pairwise ttest
df_melt = pd.melt(df, value_vars=cols)
mlp_posthoc_results = posthocs.posthoc_ttest(df_melt, val_col='value', group_col='variable', p_adjust='holm')

# Format: diagonal, non-significant, p<0.001, p<0.01, p<0.05
posthocs.sign_plot(mlp_posthoc_results, **heatmap_args)
plt.savefig('p-value.eps', format='eps', dpi=1000)